In [33]:
import numpy as np
import pydicom
from pydicom import dcmread

from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

import cv2
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pylibjpeg
import pathlib




# Import images from SALUT MS XNAT dataset.



Problems found, some images were jpeg compressed:

'pip install pylibjpeg pylibjpeg-libjpeg pydicom

TODO: 
- Process all images hmhospitales and 


### Plotting the DICOM data

To analyze our dataset, we load the paths to the DICOM files with the `get_dicom_files` function. When calling the function, we append _train/_ to the `pneumothorax_source` path to choose the folder where the DICOM files are located. We store the path to each DICOM file in the `items` list.

In [37]:
path_db ='/home/robert/data/covid/salutms_xnat'
out_path = '/home/robert/data/covid/salutms_xnat'

In [38]:
items = get_dicom_files(path_db,recurse=True)

In [39]:
print(len(items))
print(type(items))

216
<class 'fastcore.foundation.L'>


In [40]:
# trn,val = RandomSplitter(valid_pct = 0.2, seed=42)(items)

In [41]:
# print(len(trn), len(val))

Check what is CT or RX data and AP view. 

In [42]:
def check_dcm_inversion(dcm_in):
    """
    Check wether the input scan intensities are inverted or not
    based on DICOM information:
    - PresentationLUTShape == 'INVERTED'
    - PhotometricIntepretation = 'MONOCHROME'
    """
    inverted = False

    try:
        #print(dcm_in.PresentationLUTShape, end=' ')
        if dcm_in.PresentationLUTShape == 'INVERSE':
            inverted = True
    except:
        pass

    try:
        #print(dcm_in.PhotometricInterpretation, end=' ')
        if dcm_in.PhotometricInterpretation == 'MONOCHROME1':
            inverted = True
    except:
        pass

    return inverted

In [50]:
modalities = ['DX', 'CR']
def deal_DICOM(filename, folder):
    found = False
    dcm_img = pydicom.dcmread(str(filename))
    print (dcm_img.StudyDescription)
    if dcm_img.Modality in modalities:
        print ('->',filename, 'Modality', dcm_img.Modality, '- ',dcm_img.StudyDescription)
        img = dcm_img.pixel_array
        if 'WindowWidth' in dcm_img:
            print('Dataset has windowing')
        img = apply_voi_lut(dcm_img.pixel_array, dcm_img)
         # Convert to uint
        img = ((img / img.max()) * 255).astype('uint8')
        if check_dcm_inversion(dcm_img):
                img = ~img
        
        out_fn = out_path+'/'+folder+'/'+os.path.basename(filename)+'.png'
        print (out_fn)
        cv2.imwrite(out_fn,img)   
        found = True       
        # write CSV 
        p = pathlib.Path(filename)
        with open(out_path+'/salut_sms.csv', 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([p.parts[-4],os.path.basename(filename)+'.png','COVID-19'])
#        input("Press Enter to continue...")
    return found 

In [51]:
with open(out_path+'/salut_sms.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID','Filename','Class'])
found = 0
for x in items:
    if deal_DICOM(x, 'img'): found +=1
print ("end img", found)

# found = 0
# for i in trn:
#     if deal_DICOM(items[i], 'train'): found +=1
# print ("end train", found)

# found = 0
# for i in val:
#     if deal_DICOM(items[i], 'test'): found +=1
# print ("end test", found)

SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_628529_3_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200919105013322.1.345050678115-1-1-152k8eb.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200919105013322.1.345050678115-1-1-152k8eb.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_720697_1_CR_1/4/DICOM/1.2.826.0.1.3680043.2.403.84.1200919135528637.1.345050678115-4-1-1l0zg5t.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200919135528637.1.345050678115-4-1-1l0zg5t.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_348711_4_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200908154533913.1.345050678115-1-1-1txm89f.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200908154533913.1.345050678115-1-1-1txm89f.dcm.png
SALUT_M

SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_628529_2_CR_1/2/DICOM/1.2.826.0.1.3680043.2.403.84.1200919124622489.1.345050678115-2-1-unj73w.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200919124622489.1.345050678115-2-1-unj73w.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_306911_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200829144203828.1.345050678115-1-1-17m46n7.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200829144203828.1.345050678115-1-1-17m46n7.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_404260_2_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200904192214264.1.345050678115-1-1-kvexiz.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200904192214264.1.345050678115-1-1-kvexiz.dcm.png
SALUT_MS_va

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200904095122569.1.345050678115-1-1-oy2mmq.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_86042494_2_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200912092510554.1.345050678115-1-1-7x8x3t.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200912092510554.1.345050678115-1-1-7x8x3t.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/id2_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200407145849528.1.345050678115-1-1-1p4adif.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200407145849528.1.345050678115-1-1-1p4adif.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_448770_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200909082818096.1.345050678115-1-1-rhhpq9.dcm Modality CR -  SALUT_MS_valid
Dataset has windo

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200908000335758.1.345050678115-1-1-nykega.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_404260_4_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200918151455436.1.345050678115-1-1-556j4k.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200918151455436.1.345050678115-1-1-556j4k.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_672547_2_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200922143840131.1.345050678115-1-1-1m7lbg2.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200922143840131.1.345050678115-1-1-1m7lbg2.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/id6_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200402162602270.1.345050678115-1-1-19k02q5.dcm Modality CR -  SALUT_MS_valid
Dataset has window

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200917201509516.1.345050678115-1-1-1e1lvyl.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_792286_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200823054951800.1.345050678115-1-1-qx7x9s.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200823054951800.1.345050678115-1-1-qx7x9s.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_674464_4_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200924151308401.1.345050678115-1-1-4mu7ik.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200924151308401.1.345050678115-1-1-4mu7ik.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_324712_4_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200928125150611.1.345050678115-1-1-7h0zfz.dcm Modality CR -  SALUT_MS_valid
Dataset has 

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200901173837326.1.345050678115-1-1-ke2fx8.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/i456_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200629190107882.1.345050678115-1-1-q5u8si.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200629190107882.1.345050678115-1-1-q5u8si.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_628529_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200917194154773.1.345050678115-1-1-vkpxoq.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200917194154773.1.345050678115-1-1-vkpxoq.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_330988_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200920113343665.1.345050678115-1-1-dv89yr.dcm Modality CR -  SALUT_MS_valid
Dataset has windowin

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200907163409867.1.345050678115-1-1-14binsk.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_672402_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200908230019561.1.345050678115-1-1-1yiy2br.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200908230019561.1.345050678115-1-1-1yiy2br.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_357055_3_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200920095724707.1.345050678115-1-1-lrwkvq.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200920095724707.1.345050678115-1-1-lrwkvq.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_313357_2_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200922210306420.1.345050678115-1-1-183min7.dcm Modality CR -  SALUT_MS_valid
Dataset h

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200915075551834.1.345050678115-1-1-1hn3img.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_420938_4_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200831175539244.1.345050678115-1-1-riw3v2.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200831175539244.1.345050678115-1-1-riw3v2.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_348711-1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200827172525998.1.345050678115-1-1-ni0b5e.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200827172525998.1.345050678115-1-1-ni0b5e.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_857602_1_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200922112351831.1.345050678115-1-1-5tsryt.dcm Modality CR -  SALUT_MS_valid
Dataset has 

Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200914195440519.1.345050678115-1-1-1qe694.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_497076_3_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200918204353896.1.345050678115-1-1-6p21bs.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200918204353896.1.345050678115-1-1-6p21bs.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_404260_3_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200917162619654.1.345050678115-1-1-pfj8vc.dcm Modality CR -  SALUT_MS_valid
Dataset has windowing
/home/robert/data/covid/salutms_xnat/img/1.2.826.0.1.3680043.2.403.84.1200917162619654.1.345050678115-1-1-pfj8vc.dcm.png
SALUT_MS_valid
-> /home/robert/data/covid/salutms_xnat/Os_443536_6_CR_1/1/DICOM/1.2.826.0.1.3680043.2.403.84.1200926133504384.1.345050678115-1-1-77zylz.dcm Modality CR -  SALUT_MS_valid
Dataset has w